# Licence Place Detection and Extraction with Deep Learning model


This module is used to detect the presence of a licence plate in the photo and extract it.
In this module, we used a pretrained model that can detect the presence of the tunisian licence plate with the weights file named "lapi.weights".

Use case:

- The main objective of this module is to extract the licence plate (LP) from the input image and to send a cropped image of it to the module that will recognise its characters.

- The second objective is to prepare a dataset containing the licence plates cropped for the data enrichment module that is necessary to improve the model training.


!!! Note: You should verify every path in this file before using it.


## Import of necessary libraries

In [4]:
import argparse
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy import argmax,uint8
import os.path
from cv2.dnn import readNetFromDarknet,DNN_BACKEND_OPENCV,DNN_TARGET_CPU,NMSBoxes,blobFromImage
from cv2 import imwrite,rectangle,FILLED,putText,FONT_HERSHEY_SIMPLEX,getTextSize,VideoCapture,VideoWriter,VideoWriter_fourcc,CAP_PROP_FRAME_WIDTH,waitKey,getTickFrequency
import glob
from pathlib import Path

## 1- Licence Plate Detection - YOLO : Deep Learning object detection architecture

In order to detect licence we will use Yolo ( You Only Look Once ) deep learning object detection architecture 
based on convolution neural networks.
This architecture was introduced by Joseph Redmon , Ali Farhadi, Ross Girshick and Santosh Divvala first version in 2015 and later version 2 and 3.

Yolo v1 : Paper [link](https://arxiv.org/pdf/1506.02640.pdf).

Yolo v2 : Paper [link](https://arxiv.org/pdf/1612.08242.pdf).

Yolo v3 : Paper [link](https://arxiv.org/pdf/1804.02767.pdf).

Yolo is a single network trained end to end to perform a regression task predicting both object bounding box and object class.
This network is extremely fast, it processes images in real-time at 45 frames per second. A smaller version of the network, tiny YOLO, processes an astounding 155 frames per second.

You will find more information about how to train Yolo on your customized dataset in this [Link](https://towardsdatascience.com/automatic-license-plate-detection-recognition-using-deep-learning-624def07eaaf).

There is also other Deep learning object detector that you can use such as Single Shot Detector (SSD) and Faster RCNN.


In [ ]:
# Initialize the parameters
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

inpWidth = 416  #608     #Width of network's input image
inpHeight = 416 #608     #Height of network's input image

directory="D:\\Horizop_version_1.0_LP_recog\\Horizop_version_1.0_LP_recog\\"

"""

#If the image is given as a parameter via the command line $python script.py --image=path 

parser = argparse.ArgumentParser(description='Licence Plate Detection using YOLO in OPENCV')
parser.add_argument('--image', help='Path to image file.')
#parser.add_argument('--video', help='Path to video file.') #this case is not defined in the function
args = parser.parse_args()


"""
# Load names of classes
classesFile = "D:\\Horizop_version_1.0_LP_recog\\Horizop_version_1.0_LP_recog\\Licence_plate_detection\\classes.names";

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.

modelConfiguration = directory+"Licence_plate_detection\\darknet-yolov3.cfg";
modelWeights = directory+"Licence_plate_detection\\lapi.weights";

net = readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(DNN_BACKEND_OPENCV)
net.setPreferableTarget(DNN_TARGET_CPU)

In [108]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [109]:
# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom,frame):
    # Draw a bounding box.
    #    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    global LP_extracted
    LP_extracted=frame[top+6:bottom-6, left+6:right-6]
    imwrite(directory+"Licence_Plate_extracted.jpg",LP_extracted)     	#extracting the licence plate
    
    rectangle(frame, (left, top), (right, bottom), (128, 190, 82), 3) #141, 214, 88
    label = '%.2f' % conf

    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = getTextSize(label, FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])

    rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (128, 190, 82), FILLED)
    #cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine),    (255, 255, 255), cv.FILLED)
    putText(frame, label, (left, top), FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)
    #return(LP_extracted)

In [160]:

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        #print("out.shape : ", out.shape)
        for detection in out:
            #if detection[4]>0.001:
            scores = detection[5:]
            classId = argmax(scores)
            #if scores[classId]>confThreshold:
            confidence = scores[classId]
            #if detection[4]>confThreshold:
                #print(detection[4], " - ", scores[classId], " - th : ", confThreshold)
                #print(detection)
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    #print("indices",type(indices))
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height,frame)
        #print("i=",i)
        #print("box=",box)
        #PLicence=drawPred(classIds[i], confidences[i], left, top, left + width, top + height,frame)
    return(top) #added to know where we will put the text in the final image


In [177]:
def LP_detection(image):
    
    """
    :param:
    the path of the vehicle image
    
    :return:
    * if the licence plate exists in the photo, this function returns these parameters:
        - LP_extracted : is the photo of the licence plate cropped 
        - frame.astype(uint8) : a new photo same as the input photo but containing a green box showing the presence of the LP 
        and indicating the score of the detection.
        - top: is the top coordinate of the box, used in the recognition file to write on the photo the result of the licence plate
        recognition.
        
    * else: this function returns None, None, None, which should be tested in the main script to interrupt the process before 
    the recognition.
        
    """
    
    if (image is not None):
        print(image)
        cap = VideoCapture(image)
        hasFrame, frame = cap.read()
        
        # Create a 4D blob from a frame. 
        blob = blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(getOutputsNames(net))

        # Remove the bounding boxes with low confidence
        #postprocess(frame, outs)
        try:
            top=postprocess(frame, outs)

            # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
            t, _ = net.getPerfProfile()
            label = 'Inference time: %.2f ms' % (t * 1000.0 / getTickFrequency())
            #cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))


            return LP_extracted ,frame.astype(uint8),top
        except:
            return None,None,None   #in this case, we don't have a licence plate in the photo

In [184]:
#Test of the LP extraction
img= 'C:\\Users\\PC\\Desktop\\win2.png'
a,b,c=LP_detection(img)
if(a is not None and b is not None and c is not None):
    cv2.imwrite("C:\\Users\\PC\\Desktop\\extracted.jpg",a)
else:
    print("no lp")

C:\Users\PC\Desktop\win2.png
indices <class 'numpy.ndarray'>


# 2- Extracting licence plates to prepare the dataset

We can use the script to prepare a dataset containing the licence plates.

In [ ]:
vehicle_folder="D:\\license_plates_detection_train\\license_plates_detection_train" #the folder containing the vehicles
LP_folder="D:\\datasets\\LP_TN" #the output folder containing the LP extracted 

def extracting_LP(vehicle_folder,LP_folder):
    
    """
    :input: the folder containing the vehicles photos

    :output: the folder that contains the result of cropping
    
    """
    
    number_of_items=len(glob.glob("{}\\*".format(LP_folder)))+1
    
    for imagepath in glob.glob("{}\\*.*".format(vehicle_folder)):
        a,b,c=LP_detection(imagepath)
            
        if(a is not None and b is not None and c is not None):
            cv2.imwrite("{}\\{}.jpg".format(LP_folder,number_of_items),a)
            number_of_items+=1
        else: pass
extracting_LP(vehicle_folder,LP_folder)

In [ ]:
vehicle_folder="D:\\datasets\\CAMERA ANPR COPY\\20160701\\" #the folder containing the vehicles
LP_folder="D:\\datasets\\LP_TN" #the output folder containing the LP extracted 

def extracting_LP_nested_folders(vehicle_folder,LP_folder):
    """
    :input: the folder containing the folders containing the vehicles photos

    :output: the folder that contains the result of cropping
    
    In this function, we are doing the same thing as above but with nested folders like this example :
    
    ___ vehicle_folder:
        
        ______images000 :
    
            __________img1.jpg
            ...
            __________img200.jpg

        ______images001
        ...
        ______images048

    """  
    
    number_of_items=len(glob.glob("{}\\*".format(LP_folder)))+1
    #j=len(glob.glob("{}\\*".format(vehicle_folder)))
    N=0
    if(N<10):
        ch="0"+str(N)
    else:
        ch=str(N)
    while(N<=48):
        try:
            for imagepath in glob.glob("{}\\{}\\*.*".format(vehicle_folder,"images0"+ch)):
                a,b,c=LP_detection(imagepath)

                if(a is not None and b is not None and c is not None):
                    cv2.imwrite("{}\\{}.jpg".format(LP_folder,number_of_items),a)
                    number_of_items+=1
                else: pass
        except: pass
        N+=1
extracting_LP_nested_folders(vehicle_folder,LP_folder)